# SENTIMENT PREPARATION

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('polarizedTweets2018.csv')
df2 = pd.read_csv('polarizedTweets2019 .csv')
df3 = pd.read_csv('polarizedTweets2020.csv')
df3 = df3[['Date Created', 'Tweet', 'cleaned', 'label', 'Polarity']]
df3.rename(columns={'Date Created':'Date'}, inplace=True)
df4 = pd.read_csv('polarizedTweets2021.csv')
df5 = pd.read_csv('polarizedTweets2022.csv')

In [16]:
stm = pd.concat([df1,df2,df3,df4,df5], axis=0)
stm = stm[['Date','cleaned','label']]
stm['cleaned'] = stm['cleaned'].astype(str)
stm['Date'] = stm['Date'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')

stm.label.replace("positive" , 1 , inplace = True)
stm.label.replace("negative" , -1 , inplace = True)
stm.label.replace("neutral" , 0 , inplace = True)
stm.head()

,Date,cleaned,label
0,2018-12-31,aapl buy break target tradingview,0
1,2018-12-31,real reason soft apple iphone demand painfully...,-1
2,2018-12-31,spy qqq dia gdp economy aapl googl,0
3,2018-12-31,group instrument stock etf's hourly chart upda...,0
4,2018-12-31,group instrument hourly chart update member vi...,0


In [17]:
positif = []

for loc in stm['label']:
    if loc in [1]:
        positif.append(1)
    else:
        positif.append(0)

stm['positif'] = positif

In [18]:
negatif = []

for loc in stm['label']:
    if loc in [-1]:
        negatif.append(1)
    else:
        negatif.append(0)

stm['negatif'] = negatif

#### Sentimen Negatif

In [19]:
neg = stm[['Date', 'negatif']]
t3=neg.groupby(['Date', 'negatif']).size().to_frame(name='neg').reset_index()

temp = []
for i in t3.index:
    if t3['negatif'][i]==0:
        temp.append(0)
    else:
        temp.append(t3['neg'][i])
t3['neg']=temp

t3['Date'] = pd.to_datetime(t3['Date'], infer_datetime_format=True)
t3 = t3.groupby(['Date'])[['negatif','neg']].sum()

#### Sentimen Positif

In [20]:
pos = stm[['Date', 'positif']]
t2=pos.groupby(['Date', 'positif']).size().to_frame(name='pos').reset_index()

temp = []
for i in t2.index:
    if t2['positif'][i]==0:
        temp.append(0)
    else:
        temp.append(t2['pos'][i])
t2['pos']=temp

t2['Date'] = pd.to_datetime(t2['Date'], infer_datetime_format=True)
t2 = t2.groupby(['Date'])[['positif','pos']].sum()

sentimen = pd.concat([t2,t3], axis=1)

# DATASET PROCESSING

In [8]:
final_df_2018 = pd.read_csv("Final_apple_stock_2018.csv")
final_df_2019 = pd.read_csv("Final_apple_stock_2019.csv")
final_df_2020 = pd.read_csv("Final_apple_stock_2020.csv")
final_df_2021 = pd.read_csv("Final_apple_stock_2021.csv")
final_df_2022 = pd.read_csv("Final_apple_stock_2022.csv")

In [9]:
list_file = [final_df_2018, final_df_2019, final_df_2020, final_df_2021, final_df_2022]
full = pd.concat(list_file, axis=0)

In [10]:
full['Date'] = pd.to_datetime(full['Date'], infer_datetime_format=True)
full.set_index('Date', inplace=True)
full

,Open,High,Low,Close,Adj Close,Volume,P_mean,P_sum,twt_count
Date,,,,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,40.950489,102223600,-0.093750,-3,32
2018-01-03,43.132500,43.637501,42.990002,43.057499,40.943367,118071600,-0.058824,-3,51
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.133533,89738400,-0.176471,-6,34
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.601860,94640000,-0.068966,-2,29
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.447346,82271200,-0.086957,-4,46
...,...,...,...,...,...,...,...,...,...
2022-12-12,142.699997,144.500000,141.059998,144.490005,144.490005,70462700,-0.066667,-4,60
2022-12-13,149.500000,149.970001,144.240005,145.470001,145.470001,93886200,-0.039474,-3,76
2022-12-14,145.350006,146.660004,141.160004,143.210007,143.210007,82291200,-0.046875,-3,64


In [22]:
final_full = full.join(sentimen, on='Date', how='inner')
final_full.drop(['positif','negatif'], axis=1, inplace=True)
final_full['neutral'] = final_full['twt_count'] - final_full['pos'] - final_full['neg']
final_full

,Open,High,Low,Close,Adj Close,Volume,P_mean,P_sum,twt_count,pos,neg,neutral
Date,,,,,,,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,40.950489,102223600,-0.093750,-3,32,1,4,27
2018-01-03,43.132500,43.637501,42.990002,43.057499,40.943367,118071600,-0.058824,-3,51,3,6,42
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.133533,89738400,-0.176471,-6,34,1,7,26
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.601860,94640000,-0.068966,-2,29,4,6,19
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.447346,82271200,-0.086957,-4,46,2,6,38
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-12,142.699997,144.500000,141.059998,144.490005,144.490005,70462700,-0.066667,-4,60,0,4,56
2022-12-13,149.500000,149.970001,144.240005,145.470001,145.470001,93886200,-0.039474,-3,76,3,6,67
2022-12-14,145.350006,146.660004,141.160004,143.210007,143.210007,82291200,-0.046875,-3,64,2,5,57


In [23]:
# Saving Dataset to csv
final_full.to_csv('Final_apple_stock_twit.csv')